In [18]:
import pandas as pd
import numpy as np
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
from sklearn import cluster
from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import kneighbors_graph
from statsmodels.tsa.stattools import acf
from statsmodels.tsa.stattools import pacf
from scipy.signal import periodogram

Fourier Transform

In [23]:
load = pd.read_csv('Load_history.csv')

In [24]:
load2 = load.dropna()

In [26]:
hour = ['h1', 'h2', 'h3', 'h4','h5', 'h6', 'h7', 'h8','h9', 'h10', 'h11', 'h12','h13', 'h14', 'h15', 'h16','h17', 'h18', 'h19', 'h20','h21', 'h22', 'h23', 'h24']

In [27]:
load = load2[hour]

In [28]:
da1 = np.array(load[0:1])
da2 = np.array(load2[0:1])
da1 = normalize(da1)

In [29]:
fr, pe = periodogram(da1)
pe = normalize(pe)
a = np.concatenate((da2[0], pe[0]))
data = [a]
for i in range(len(load)):
    da1 = np.array(load[i+1 : i+2])
    da2 = np.array(load2[i+1 : i+2])
    da1 = normalize(da1)
    fr, pe = periodogram(da1)
    pe = normalize(pe)
    da = np.concatenate((da2[0], pe[0]))
    data = np.concatenate((data, [da]))

In [30]:
data = pd.DataFrame(data)

In [31]:
data = data.rename(columns = {0 : 'zone_id', 1 : 'year', 2:'month', 3 : 'day'})

In [32]:
d = {}
for i in range(4, 28):
    d[i] = i - 3
d1 = {28 : 'f1', 29:'f2', 30:'f3', 31:'f4', 32:'f5', 33:'f6', 34:'f7', 35:'f8', 36:'f9', 37:'f10', 38:'f11', 39:'f12', 40:'f13'}

In [33]:
data = data.rename(columns = d)
data = data.rename(columns = d1)

In [34]:
data = data.rename(columns = {'h13':13,'h14':14,'h15':15,'h16':16,'h17':17,'h18':18,'h19':19, 'h20':20,'h21':21,'h22':22,'h23':23,'h24':24})

In [35]:
aa = pd.melt(data, id_vars=[u'zone_id', u'year', u'month', u'day', u'f1', u'f2', u'f3', u'f4', u'f5', u'f6', u'f7', u'f8',
       u'f9', u'f10', u'f11', u'f12', u'f13'],
             value_vars=[1,          2,
                3,          4,          5,          6,          7,          8,
                9,         10,         11,         12,         13,         14,
               15,         16,         17,         18,         19,         20,
               21,         22,         23,         24], var_name='hours', value_name='load')
res = aa.sort_values([u'zone_id', u'year', u'month', u'day'], ascending=[1, 1, 1, 1])

In [ ]:
res.to_csv('fredata.csv', index = False)

Time Series Clustering

In [36]:
load = pd.read_csv('res.csv')
loaddata = load[['zone_id', 'load']]
loaddata1 = np.array(loaddata[loaddata['zone_id'] == 1].drop('zone_id', axis = 1))
loaddata1 = loaddata1[~np.isnan(loaddata1)]
data = [loaddata1.reshape(loaddata1.shape[0],)]

In [37]:
for i in range(19):
    i = i+2
    d = np.array(loaddata[loaddata['zone_id'] == i].drop('zone_id', axis = 1))
    d = d[~np.isnan(d)]
    da = [d.reshape(d.shape[0],)]
    data = np.concatenate((data, da), axis = 0)

In [38]:
X = StandardScaler().fit_transform(data)
ncluster = 3

In [40]:
spectral = cluster.SpectralClustering(n_clusters=ncluster,
                                          eigen_solver='arpack',
                                          affinity='rbf')
two_means = cluster.MiniBatchKMeans(n_clusters=ncluster)
ward = cluster.AgglomerativeClustering(n_clusters=ncluster, linkage='ward',
                                           connectivity=connectivity)
birch = cluster.Birch(n_clusters=ncluster)
affinity_propagation = cluster.AffinityPropagation(damping=.9,
                                                       preference=-200)

In [41]:
clustering_algorithms = [two_means, affinity_propagation, spectral, ward, birch]

In [53]:
data_acf = [acf(data[0])]
for i in range(19):
    i = i + 1
    datai = [acf(data[i])]
    data_acf = np.concatenate((data_acf, datai), axis = 0)

In [54]:
X_acf = data_acf

In [55]:
bandwidth = cluster.estimate_bandwidth(X_acf, quantile=0.3)
 # connectivity matrix for structured Ward
connectivity = kneighbors_graph(X_acf, n_neighbors=10, include_self=False)
# make connectivity symmetric
connectivity = 0.5 * (connectivity + connectivity.T)

In [56]:
for algorithm in clustering_algorithms:
    y_pre = algorithm.fit_predict(X_acf)
    print y_pre

[1 1 1 2 1 1 1 1 1 0 1 1 2 1 1 2 1 1 1 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1 1 1 2 1 1 1 0 0 0 0 1 2 0 0 2 0 0 0 0]
[0 0 0 0 0 0 0 0 2 1 0 0 0 0 0 0 0 0 0 0]
[2 2 2 2 2 2 2 2 1 0 2 2 2 2 2 2 2 2 2 2]


/Users/tianyucui/anaconda/lib/python2.7/site-packages/sklearn/cluster/k_means_.py:1279: DeprecationWarning:

This function is deprecated. Please call randint(0, 19 + 1) instead

/Users/tianyucui/anaconda/lib/python2.7/site-packages/sklearn/cluster/k_means_.py:1328: DeprecationWarning:

This function is deprecated. Please call randint(0, 19 + 1) instead

/Users/tianyucui/anaconda/lib/python2.7/site-packages/sklearn/cluster/k_means_.py:1328: DeprecationWarning:

This function is deprecated. Please call randint(0, 19 + 1) instead

/Users/tianyucui/anaconda/lib/python2.7/site-packages/sklearn/cluster/k_means_.py:1328: DeprecationWarning:

This function is deprecated. Please call randint(0, 19 + 1) instead

/Users/tianyucui/anaconda/lib/python2.7/site-packages/sklearn/cluster/k_means_.py:1328: DeprecationWarning:

This function is deprecated. Please call randint(0, 19 + 1) instead

/Users/tianyucui/anaconda/lib/python2.7/site-packages/sklearn/cluster/k_means_.py:1328: DeprecationWarning:

T

In [57]:
from sklearn import manifold

In [58]:
n_neighbors = 3
n_components = 2
Y = manifold.Isomap(n_neighbors, n_components).fit_transform(X_acf)

In [61]:
Y

array([[-0.50825082,  0.07907164],
       [-0.31794995,  0.30500698],
       [-0.31795059,  0.30500665],
       [ 0.29661837, -0.70435985],
       [-0.30198491, -0.28647092],
       [-0.32228705,  0.27050235],
       [-0.31795059,  0.30500665],
       [-0.00433802, -0.07538758],
       [-2.40999913, -0.4609608 ],
       [ 2.65080219, -0.19039058],
       [ 0.17010525,  0.17330503],
       [-0.15056046,  0.22977394],
       [ 0.71376286, -0.16222178],
       [ 0.06994348,  0.18114149],
       [ 0.00905952,  0.1528749 ],
       [ 0.61249388, -0.01356386],
       [ 0.07576142,  0.13771603],
       [ 0.13702452,  0.02740626],
       [ 0.04920883, -0.00756092],
       [-0.13350879, -0.26589563]])

In [59]:
xt = Y[:, 0]
yt = Y[:, 1]

In [60]:
trace = go.Scatter(
    x = xt,
    y = yt,
    mode = 'markers'
)

data = [trace]
py.iplot(data, filename='basic-scatter')